# Ok, Google? #

As soon as number of texts, objects, records and so on becomes bigger, users start suffering from waiting while performing search queries. Computer scientist invented different tricks to speed up search. Let's explore few such techniques.

**Here's your model problem**. Your friend and you want to spend cool time playing [Jeopardy](https://en.wikipedia.org/wiki/Jeopardy!). One asks questions, other answers. You found a big database of questions. You don't what to play random questions, but only those related to some topics. So, your friend and you build a search engine that returns questions that contain **exact substring**<sup>1</sup>.

<sup>1</sup> - generally search engines don't bother about **exact** matches, as users do a lot of typos, simplifications, ommit articles. But you and your friend want to experience difference between exact and approximate search.

Anaconda holds most of requirements, but please make sure you have `requests` and `numpy` installed.

In [5]:
#! pip install --user requests
! pip install sortedcontainers

  Using cached https://files.pythonhosted.org/packages/13/f3/cf85f7c3a2dbd1a515d51e1f1676d971abe41bba6f4ab5443240d9a78e5b/sortedcontainers-2.1.0-py2.py3-none-any.whl


You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


All necessary requirements are imported here.

In [6]:
import sys, time, requests, copy
import csv, re, random, sortedcontainers # I use SortedDict at the last phase. Maybe you will also do.
import numpy as np

Here we start preparing the data. Run next 3 blocks once to load everything to RAM (yes, we will fit in RAM for this tutorial).

Firstly, load stop-words list. These are the words, that are not considered informative in index structures, as they are met in practically any text.

In [15]:
stopwords_url = "https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/"
stopwords = set(requests.get(stopwords_url).text.split())
print("Sample:", *list(stopwords)[:50])

Sample: only don its did hers or when s these above they is ourselves during once so no same yours now him very again doing most those through yourselves and of am he we how themselves where herself can more while myself this does up an should over under nor both


Now let's load questions and answers into RAM. (Actually we will not use answers in this tutorial, but maybe you will play Jeopardy indeed?). `jeop.csv` file is attached to your tutorial.

In [13]:
start = time.time()
texts = []
words = []
answers = {}
with open("jeop.csv", encoding="utf8") as csvfile:
    for line in csv.DictReader(csvfile, delimiter=',', quotechar='`'):
        question = line["Question"].lower()
        answer = (line["Answer"] or "").lower()
        words = re.findall("[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\'\w\-]+", question)
        texts.append(question)
        answers[question] = answer
        
finish = time.time()
print("{} rows loaded in {:.2f} ms".format(len(texts), (finish - start) * 1000))

216930 rows loaded in 3620.30 ms


In [20]:
type(texts[1])

str

And the last block - test texts and queries. Just to keep test simple.

In [14]:
test_texts = [
    "Who let the dogs out?",
    "Let's go, guys!",
    "Mamma mia!!!",
    "Here is the first sentence. And here is the second..."
]

test_queries = [
    "go home",
    "who said",
    "what was the"
]

## 0. Naive substring search ##

**Please write here the most naive substring search you can :)**

- `query` - substring you are looking for.
- `texts` - `list` of any other iterable collections of `string`s.
- `returns` - list of string containing `query` as a substring.

In [23]:
def search_naive(query, texts):
    result = []
    
    for text in texts:
        if text.find(query) != -1:
            result.append(text)
            
    assert all([query in text for text in result])
    return result
search_naive(test_queries[0], texts)

["`i'll go home with bonnie jean` is one of many lively songs in this lerner & loewe musical",
 "you can `go home again` to see this author's boyhood home in asheville, north carolina",
 "in the '30s wolfe wrote `you can't go home again` & kaufman & hart wrote `you can't` do this",
 'he wrote, `i come to my solitary woodland walk as the homesick go home`:',
 "<a href=`http://www.j-archive.com/media/2009-01-01_j_06.mp3`>`i won't think of it now... i'll go home to tara tomorrow`</a>",
 "he was born at home oct. 3, 1900 in asheville, n.c. & later found out you can't go home again",
 '`better be worth it.  he better go home and cure some disease or invent a longer-lasting light bulb`',
 'when nancy pelosi & maxine waters go home from the u.s. house, they go to this state',
 'a traditional variation on `show me the way to go home` is `indicate the direction of my habitual` this',
 "after the gauls besieged this city in 390 b.c., they were bought off so they'd go home",
 "win an academy awar

In [24]:
for query in test_queries:
    print("Query:", query)
    start = time.time()
    ans = search_naive(query, texts)
    finish = time.time()
    print("Time: {:.2f} ms".format(1000 * (finish - start)))
    print("Size:", len(ans))
    print("Sample:", ans[:2])
    print()

Query: go home
Time: 76.91 ms
Size: 28
Sample: ["`i'll go home with bonnie jean` is one of many lively songs in this lerner & loewe musical", "you can `go home again` to see this author's boyhood home in asheville, north carolina"]

Query: who said
Time: 61.53 ms
Size: 54
Sample: ['film character who said, `you should be kissed, and often, and by someone who knows how`', '`manhattan`ite who said, `life is divided into the horrible and the miserable`--sounds `bananas` to us']

Query: what was the
Time: 58.03 ms
Size: 16
Sample: ['on may 4, 1980 tito died in ljubljana in what was then this country', "(<a href=`http://www.j-archive.com/media/2007-02-27_dj_29.jpg` target=`_blank`>jimmy of the clue crew reports from the pentagon.</a>)  i'm at the building put up between 1941 & 1943 to house what was then this government department"]



Not bad... So fast? But this search is linear with respect to the number of text - `O(N)`. Thus, 200M texts will require 1000 times longer time. Shall you wait for 30 seconds each time?

## 1. Inverted index ##
Inverted index is a beatiful concept, used in major search engines up to 2017. For details please refer to p. 234 of "Algorithms in the real world" lecture notes.
### 1.1. Text to tokens ###
Firstly, we need to extract the grains of the text. They should not be very small (letters will not give us any profit) and not very big (almost all sentences are unique). Words are good: there are few thousands of them, which is not very big number for hash table, but still big enough to catch the difference amoung texts.

There are multiple approaches to extracting words from sentences. "Correct" approach is to use formal language model and run a parser (tokenizer) that will extract and label all the words and punktuation. "Simpler" approach (either split+strip, or regular expressions) have a benefit - they don't need any additional libraries like `nltk` :).

In [25]:
def sentence_to_words(txt):
    # Here we lower case and split the sentence into words
    result = re.findall("[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\'\w\-]+", txt.lower()) # регулярное выражение
    
    assert bool(result) == bool(txt.strip()), "Empty list for empty string only."
    if bool(result):
        assert all([w in txt.lower() for w in result]), "Tokens are substrings of the text."
    return result

Tests for splitting. Assertions should not fail, splitting should look reasonable.

In [26]:
for text in test_texts:
    print(text, "=>", sentence_to_words(text))

Who let the dogs out? => ['who', 'let', 'the', 'dogs', 'out']
Let's go, guys! => ["let's", 'go', 'guys']
Mamma mia!!! => ['mamma', 'mia']
Here is the first sentence. And here is the second... => ['here', 'is', 'the', 'first', 'sentence', 'and', 'here', 'is', 'the', 'second']


### 1.2. Remove stopwords ###
One important heuristic about indexing. There is relatively few words (say, few dozens of stop words) which are present in approximately all texts. Adding them to index will lead to growing index size and practically zero profit.

**Implement a function which removes all stop words.**

- `tokens` - list of lowercase words.
- `stopwords` - `set` of words to exclude.
- `returns` - `list` of words without stop words.

In [45]:
def clear_tokens(tokens, stopwords):
    result = []
    #TODO: write here the code which filters result list
    for token in tokens:
        if token not in stopwords:
            result.append(token)
    
    assert not any([token in stopwords for token in result]), "No stopwords in resulting list."
    return result

Please make sure your filtering works ok.

In [46]:
for text in test_texts:
    print(text, "=>", clear_tokens(sentence_to_words(text), stopwords))

Who let the dogs out? => ['let', 'dogs']
Let's go, guys! => ["let's", 'go', 'guys']
Mamma mia!!! => ['mamma', 'mia']
Here is the first sentence. And here is the second... => ['first', 'sentence', 'second']


### 1.3. Inverted index ###

And here comes index builing! **Please add necessary code that builds inverted index.**

- `texts` - `list`, holding `string`s.
- `returns` - `dict` with lowercase token (word) as key and `set` of text indices as values.

In [91]:
from collections import defaultdict

def build_inverted_index(texts):
    iindex = defaultdict(set)

    for i, text in enumerate(texts):
        for tok in clear_tokens(sentence_to_words(text), stopwords):
            #TODO: write here the code which prepares inverted index
            if tok in text:
                iindex[tok].add(i)
    
                
    assert iindex, "Empty index"
    assert "disney" in iindex, "Word is missing"
    assert iindex["disney"], "Something wrong with posting list"
    
    return iindex

Here we build inverted index for our real data and measure time.

In [92]:
start = time.time()
inverted_index = build_inverted_index(texts)
finish = time.time()
print("Inverted index build took {:.2f} ms".format(1000 * (finish - start)))

Inverted index build took 4756.64 ms


**Add here necessary code to perform search in inverted index.**

- `query` - text we search for.
- `texts` - `list` of original texts.
- `ii` - `dict` holding inverted index.
- `returns` - `list` of strings from texts, subset of texts relevant for query.

In [103]:
def search_in_inv_index(query, texts, ii):
    result = []
    q = clear_tokens(sentence_to_words(query), stopwords)
    postings = None
    lists = list()
    for iq in q:
        lists.append(ii[iq])
    if len(lists) > 0:    
        postings = lists[0]
        for l in lists:
            postings = postings.intersection(l)
        
    #TODO: find here intersection of all posting lists, corresponding to words in q
    
    result = [texts[idx] for idx in postings or [] if query in texts[idx]]
    return result

Run test code. Is it faster? Do you have the same output sets? If no, can you explain, why?

In [111]:
for query in test_queries:
    print("Query:", query)
    start = time.time()
    ans = search_in_inv_index(query, texts, inverted_index)
    finish = time.time()
    print("Time: {:.3f} ms".format(1000 * (finish - start)))
    print("Size:", len(ans))
    print("Sample:", ans[:2])
    print()

Query: go home
Time: 0.496 ms
Size: 28
Sample: ["`you're all clear, kid, now let's blow this thing & go home` is spoken just before this is destroyed in a 1977 movie", "win an academy award & you'll go home with this golden guy"]

Query: who said
Time: 1.488 ms
Size: 54
Sample: ["famous blonde of the '30s who said, `when women go wrong, men go right after them`", "it's the last name of the character who said, `every time a bell rings, an angel gets his wings`"]

Query: what was the
Time: 0.000 ms
Size: 0
Sample: []



### Because we delete all stopwords

## 2. Embedding ##
Embedding is a beatiful concept, which allows to bring completely different mathematics to search structures. If we can find a good approach for embedding some object (text, picture, voice, person, etc), then we can utilize cool tricks from linear algebra and data structures to speed up search operations.
### 2.1. Lexicon ###
Fundamental idea of [distributional semantics](https://en.wikipedia.org/wiki/Distributional_semantics) is that text meaning (semantics) is hidden in words and their close groupings (near context). Of course, this is just a hypothesis, but it was empirically "proven" by multiple useful applications.

**Please write a code, which extracts lexicon and inverted lexicon from texts.**

`texts` - `list` of texts.

`returns` 
 - `lexicon` = `list` of words (`i -> word`); 
 - `inv_lexicon` = `dict` of `lexicon`<sup>-1</sup> (`word -> i`).

In [126]:
def get_lexicon_and_inverted_lexicon(texts):
    lexicon = set()
    inv_lexicon = defaultdict()
        
    for text in texts:
        toks = clear_tokens(sentence_to_words(text), stopwords)
            # TODO: fill the lexicon list with unique words (order not important)
            # and build an inverted version
        for tok in toks:
            lexicon.add(tok)
        
    for i, tok in enumerate(lexicon):
        inv_lexicon[tok] = i
         
    if texts:
        assert len(lexicon) == len(inv_lexicon), "Container sizes should match"
    return lexicon, inv_lexicon

Let's build lexicon just once!

In [127]:
start = time.time()
Lex, iLex = get_lexicon_and_inverted_lexicon(texts)
finish = time.time()
print("Lexicon building time: {:.2f} ms".format(1000 * (finish - start)))

Lexicon building time: 3803.88 ms


If we just encode our texts in lexicon-dimensional space (TF-IDF of just 0-s and 1-s), we will need 200K vectors of 123K of `float`s. Just to hold this particular matrix, we will have to move to hard drive. Let's avoid this in our tutorial :) 

PS But PCA still works great for dimensionality reduction!

In [128]:
print("To store matrix we need {} * {} = {}B floats".format(len(texts), len(Lex), int(len(texts) * len(Lex) // 1e9)))

To store matrix we need 216930 * 122917 = 26B floats


### 2.2. Embedding ###
Yes, creating 123K-dimensional vector for each text was very bad idea. Let's use one of smart techniques. I will bring some false positives (some non-related texts will become "similar"), but still keeps the idea. This method allows easily tune embedding size to balance between quality and memory.

- `text` - text to embed.
- `inv_lexicon` - inverted lexicon `dict`.
- `dim` - resulting vector size.
- `returns` - `dim`-dimensional normed vector (1-dimensional `np.ndarray`)

In [129]:
def embed(text, inv_lexicon, dim=100):
    result = np.zeros((dim,))
    for tok in clear_tokens(sentence_to_words(text), stopwords):
        if tok in inv_lexicon:
            result[inv_lexicon[tok] % dim] = 1
    norm = np.linalg.norm(result)
    if norm > 0.0000001:  # yes, I know, magic number :(
        result /= norm
    return result

Here we build a matrix `M` with all embedding aligned in rows.

In [130]:
D = 100

start = time.time()
M = np.zeros((len(texts), D))
for i, text in enumerate(texts):
    M[i, :] = embed(text, iLex)
    
finish = time.time()
print("Matrix building time: {:.2f} ms".format(1000 * (finish - start)))

Matrix building time: 6733.20 ms


In [131]:
M

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.31622777],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.30151134, ..., 0.        , 0.        ,
        0.        ]])

### 2.3. Search (ANNS - approximate nearest neighbour search) ###
As soon as embedding results with a normed vector, cosine similarity equals dot product. Thus, we can multiply `M` by an embedding of query to get a vector of similarities. Here we can use a technique discussed in lecture: pre-ranking set. We take only `top` most similar texts, and check only them for exact match. 

**Complete the code that performs search.**

- `query` - search query.
- `texts` - `list` of all texts.
- `M` - `|TEXTS|*D` text embedding matrix.
- `iLex` - inverted lexicon `dict`.
- `top` - size of pre-ranking set.
- `returns` - `list` of relevant texts.

In [132]:
def search_by_dot_product(query, texts, M, iLex, top=5000):
    result = []
    vec = embed(query, iLex)
    texts = np.array(texts)
    result = texts[np.argsort(M @ vec.transpose())[::-1][:top]]
    result = [t for t in result if query in t]
    
    assert all([query in t for t in result]), "Results should match query!"
    return result

Did the result of search change? Why? Can we influence this result without rewriting search code? Without rebuilding index matrix `M`?

In [133]:
for query in test_queries:
    print("Query:", query)
    start = time.time()
    ans = search_by_dot_product(query,texts, M, iLex)
    finish = time.time()
    print("Time: {:.3f} ms".format(1000 * (finish - start)))
    print("Size:", len(ans))
    print("Sample:", ans[:2])
    print()

Query: go home
Time: 624.461 ms
Size: 28
Sample: ["`you can't go home again` until you name this author who wrote it", "the summers are nice in this capital of manitoba--that's when i should have come. me wanna go home"]

Query: who said
Time: 556.511 ms
Size: 6
Sample: ['film character who said, `you should be kissed, and often, and by someone who knows how`', "character who said, `i'm not bad.  i'm just drawn that way`"]

Query: what was the
Time: 524.768 ms
Size: 0
Sample: []



## 3. Small world network ##
We discussed [small world networks](https://en.wikipedia.org/wiki/Small-world_network) in lecture. This beautiful concepts utilize skip-list idea to reach query neighbourhood fastly from any random graph node. You have practically all code written, you just need to call `rewire()` function with respect to [Watts–Strogatz process](https://en.wikipedia.org/wiki/Watts%E2%80%93Strogatz_model).

**Please write rewiring code.**

Function `build_graph` accepts some iterable collection of `values`. In our case this will be embeddings. 

- `K` is a parameter of Watts–Strogatz model, expressing average degree of graph nodes.
- `p` stands for probability of "rewiring".

In [154]:
class Node:
    ''' Graph node class. Major properties are `value` to access embedding and `neighbourhood` for adjacent nodes '''
    def __init__(self, value, idx):
        self.value = value
        self.idx = idx
        self.neighbourhood = set()
        

def build_graph(values, K, p=0.4):
    '''Accepts container with values. Returns list with graph nodes'''
    
    def rewire(nodes, i, j, k):
        ''' removes i-j connection and adds i-k connection, bi-directional'''
        
        # trivial case and loop
        if j == k or i == k: return False
        # parallel edges
        if k in nodes[i].neighbourhood: return False
        
        nodes[i].neighbourhood.remove(j)
        nodes[j].neighbourhood.remove(i)
        nodes[k].neighbourhood.add(i)
        nodes[i].neighbourhood.add(k)
        return True
    
        
    N = len(values)
    nodes = [None] * N
    
    # create nodes
    for i, val in enumerate(values):
        nodes[i] = Node(val, i)
    
    # create K-regular lattice
    for i, val in enumerate(nodes):
        for j in range(i - K // 2, i + K // 2 + 1):
            if i != j:
                nodes[i].neighbourhood.add(j % N)
                nodes[j % N].neighbourhood.add(i)
    
    for i, node in enumerate(nodes):
        for j in range(i - K // 2, i):
            # TODO: for each node i rewire right hand side i-j edge to some other random node k with probability p
            # See Watts–Strogatz model for details
            
            k = np.random.randint(0, len(nodes)-1)
            rewire(nodes, i, j % N, k) 

#             if i != j:
#                 k = np.random.randint(i,len(nodes))
#             if i != k:
#                 nodes[i].neighbourhood.add(k % N)
#                 nodes[k % N].neighbourhood.add(i)
            pass
                
    return nodes

The bigger `K` and `p` you choose, the longer method runs. Bigger `K` leads to bigger near-cliques in a graph and, as a consequence, bigger context to consider at each step of search. Bigger `q` is for a better "remove hops", but it should not be close to 1, as it will make graph random (not SW). If you use next block to test previous block, **PLEASE** use small numbers (e.g. `K=10`, `p=0.2`). For final search index use bigger `K` (`K=500`, `p=0.3` works for 90-300 sec. on my machine).

In [155]:
start = time.time()
G = build_graph(M[:19000,:], K=500, p=0.3)
finish = time.time()
print("Graph built in {:.2f} ms".format(1000 * (finish - start)))

Graph built in 21240.21 ms


And the last step left. We need to implement efficient search procedure which utilize small world properties. You should move current node towards query embedding, keeping fresh nearest neightbours collection. **Please implement basic NSW search**. You can refer to an algorithm at the bottom of page 3 in this [original paper](https://publications.hse.ru/mirror/pubs/share/folder/x5p6h7thif/direct/128296059). Note, that pre-ranking set size will most probably not influence search speed.

`search_nsw_basic()`
- `query` - `vector` (`np.ndarray`) representing a query.
- `nsw` - SW graph.
- `top` - re-ranking set size.
- `guard_hops` - if method does not converge, we will terminate it.
- `returns` - a pair of a `set` of indices and number of hops `(nearest_neighbours_set, hops)`

`search_nsw()`
- `query` - text query.
- `texts` - `list` of texts.
- `nsw` - NSW graph.
- `iLex` - inverted lexicon.
- `returns` - a pair of a `list` of relevant texts and number of hops `(relevant_texts, hops)`

In [ ]:
def search_nsw_basic(query, nsw, top=5000, guard_hops=64):
    ''' basic algorithm, takes vector query and returns a pair (nearest_neighbours, hops)'''
    
    # I used this in my solution. 
    # They can do .peekitem() and popitem() to access biggest key
    nn = sortedcontainers.SortedDict()
    to_visit = sortedcontainers.SortedDict()
    visited = set()
    current = random.randint(0, len(nsw) - 1)
    hops = 0
    
    # we will break the loop when converged, but we also guard for the case
    while hops < guard_hops:
        hops += 1
        visited.add(current)
        sim = np.dot(query, nsw[current].value)
        
        # TODO: write here you cool code!!!
        # I will store here updated sorted list of closes nodes
    
    # next 4 lines are the part of my solution. You can remove or reuse them
    result = set()
    while nn and top > 0:
        top -= 1
        result.add(nn.popitem()[1])

    return result, hops

        
def search_nsw(query, texts, nsw, iLex):
    query_embedding = embed(query, iLex)
    nn_indices, hops = search_nsw_basic(query_embedding, nsw)
    result = [texts[j] for j in nn_indices if query in texts[j]]
    return result, hops

Most probably your solution will run for some hundreds of milliseconds. But please don't become upset! First of all, this is because this is python. Secondly, you could choose non-optimal parameters or non-optimal data structures for function internal containers (so did I). Third, we used awful embedding function :). And the last but not the least: `O(K×D×log(N))`. Thus, for 200M records your solution will still fit in a second!

In [ ]:
for query in test_queries:
    print("Query:", query)
    start = time.time()
    ans, hops = search_nsw(query, texts, G, iLex)
    print("Hops:", hops)
    finish = time.time()
    print("Time: {:.3f} ms".format(1000 * (finish - start)))
    print("Size:", len(ans))
    print("Sample:", ans[:2])
    print()